# HW Lesson_4

### 1). 
#### скачать набор данных маркетинговых кампаний отсюда https://www.kaggle.com/davinwijaya/customer-retention

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import catboost

#### Marketing Promotion Campaign Uplift Modelling

- **recency** - months since last purchase
- **history** - value of the historical purchases
- **used_discount** - indicates if the customer used a discount before
- **zip_code** - class of the zip code as Suburban/Urban/Rural
- **is_referral** - inicates if the custormer was acquired from referral channel
- **channel** - channels that the customer using. Phone/Web/Multichannel
- **offer** - the offers sent to the customers, Discount/But One Get One/No Offer
- **conversion** - customer conversion (buy or not)

In [2]:
df_data = pd.read_csv('data.csv')

In [3]:
df_data

recency  history  used_discount  used_bogo   zip_code  is_referral  \
0           10   142.44              1          0  Surburban            0   
1            6   329.08              1          1      Rural            1   
2            7   180.65              0          1  Surburban            1   
3            9   675.83              1          0      Rural            1   
4            2    45.34              1          0      Urban            0   
...        ...      ...            ...        ...        ...          ...   
63995       10   105.54              1          0      Urban            0   
63996        5    38.91              0          1      Urban            1   
63997        6    29.99              1          0      Urban            1   
63998        1   552.94              1          0  Surburban            1   
63999        1   472.82              0          1  Surburban            0   

            channel            offer  conversion  
0             Phone  Buy One Get One           0  
1               Web         No Offer           0  
2               Web  Buy One Get One           0  
3               Web         Discount           0  
4               Web  Buy One Get One           0  
...             ...              ...         ...  
63995           Web         Discount           0  
63996         Phone         Discount           0  
63997         Phone         Discount           0  
63998  Multichannel  Buy One Get One           0  
63999           Web         Discount           0  

[64000 rows x 9 columns]

### 2). 
#### поле conversion - это целевая переменная, а offer - коммуникация. Переименовать поля (conversion -> target, offer -> treatment) и привести поле treatment к бинарному виду (1 или 0, т.е было какое-то предложение или нет) - значение No Offer означает отсутствие коммуникации, а все остальные - наличие.

In [4]:
df_data.rename(columns={
    'offer': 'treatment',
    'conversion': 'target'
}, inplace=True)

In [5]:
df_data.loc[df_data['treatment'] == 'No Offer', 'treatment'] = 0
df_data.loc[df_data['treatment'] != 0, 'treatment'] = 1

In [6]:
df_data['treatment'] = df_data['treatment'].astype(int)

In [7]:
df_data['treatment'].dtype

dtype('int32')

In [8]:
y = df_data['target']
X = df_data.drop(columns='target')

### 3).  
#### сделать разбиение набора данных не тренировочную и тестовую выборки

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=True, stratify=y, random_state=15)

In [10]:
X_train.iloc[:5]

recency  history  used_discount  used_bogo   zip_code  is_referral  \
23036        1   526.72              1          0      Urban            1   
53943        9   670.23              0          1  Surburban            1   
16392        5   118.70              0          1      Rural            1   
60643        2    97.11              1          0      Urban            0   
8809         1  1139.94              1          1      Urban            1   

      channel  treatment  
23036     Web          1  
53943     Web          1  
16392   Phone          1  
60643   Phone          0  
8809    Phone          1

In [11]:
treat_train = X_train['treatment']
treat_test = X_test['treatment']

In [12]:
X_train = X_train.drop(columns='treatment')
X_test = X_test.drop(columns='treatment')

### 4). 
#### провести uplift-моделирование 3 способами:
- одна модель с признаком коммуникации (S-learner)
- модель с трансформацией таргета
- вариант с двумя независимыми моделями

In [13]:
models_results = {
    'approach': [],
    'uplift@10%': [],
    'uplift@20%': [],
    'uplift@30%': [],
    'uplift@50%': []
}

**Solo model**

In [14]:
from sklift.models import SoloModel
from catboost import CatBoostClassifier
from sklift.metrics import uplift_at_k
from sklift.viz import plot_uplift_curve

In [15]:
cat_features = ['zip_code', 'channel']

In [16]:
sm = SoloModel(
    CatBoostClassifier(iterations=100, random_state=15,
                       silent=True, cat_features=cat_features)
)

In [17]:
sm.fit(X_train, y_train, treat_train)

SoloModel(estimator=<catboost.core.CatBoostClassifier object at 0x000002C871C608B0>)

In [18]:
uplift_sm = sm.predict(X_test)

In [19]:
# функция внесения результата модели в словарь models_results
def score(model_pred, k):
    sm_score_k = uplift_at_k(y_true=y_test, uplift=model_pred,
                           treatment=treat_test, strategy='by_group',
                           k=k)
    return sm_score_k

In [20]:
models_results['approach'].append('SoloModel')
models_results['uplift@10%'].append(score(uplift_sm, 0.1))
models_results['uplift@20%'].append(score(uplift_sm, 0.2))
models_results['uplift@30%'].append(score(uplift_sm, 0.3))
models_results['uplift@50%'].append(score(uplift_sm, 0.5))

**Transformation class**

In [21]:
from sklift.models import ClassTransformation

In [22]:
ct = ClassTransformation(
    CatBoostClassifier(iterations=100, random_state=15,
                       silent=True, cat_features=cat_features)
)

In [23]:
ct.fit(X_train, y_train, treat_train)
uplift_ct = ct.predict(X_test)

In [24]:
models_results['approach'].append('ClassTransformation')
models_results['uplift@10%'].append(score(uplift_ct, 0.1))
models_results['uplift@20%'].append(score(uplift_ct, 0.2))
models_results['uplift@30%'].append(score(uplift_ct, 0.3))
models_results['uplift@50%'].append(score(uplift_ct, 0.5))

In [25]:
models_results

{'approach': ['SoloModel', 'ClassTransformation'],
 'uplift@10%': [0.07679603431914267, 0.06745877824047308],
 'uplift@20%': [0.07077198822173858, 0.05830300370052002],
 'uplift@30%': [0.06875946682618209, 0.05473562783056579],
 'uplift@50%': [0.0641619235553319, 0.05762591369804701]}

**Two independed models**

In [26]:
from sklift.models import TwoModels

In [27]:
tm = TwoModels(
    CatBoostClassifier(iterations=100, random_state=15,
                       silent=True, cat_features=cat_features),
    CatBoostClassifier(iterations=100, random_state=15,
                       silent=True, cat_features=cat_features)
)

In [28]:
tm = tm.fit(X_train, y_train, treat_train)

In [29]:
uplitf_tm = tm.predict(X_test)

In [30]:
models_results['approach'].append('TwoModels')
models_results['uplift@10%'].append(score(uplitf_tm, 0.1))
models_results['uplift@20%'].append(score(uplitf_tm, 0.2))
models_results['uplift@30%'].append(score(uplitf_tm, 0.3))
models_results['uplift@50%'].append(score(uplitf_tm, 0.5))

### 5). 
#### в конце вывести единую таблицу сравнения метрик uplift@10%, uplift@20% 3 моделей

In [31]:
pd.DataFrame(data=models_results)

approach  uplift@10%  uplift@20%  uplift@30%  uplift@50%
0            SoloModel    0.076796    0.070772    0.068759    0.064162
1  ClassTransformation    0.067459    0.058303    0.054736    0.057626
2            TwoModels    0.051746    0.069969    0.068971    0.064731

### 6).
#### *для модели S-learner построить зависимость таргета (конверсии - поле conversion) от значения uplift:
- сделать прогноз и получить uplift для тестовой выборки
- отсортировать тестовую выборку по uplift по убыванию
- разбить на децили (pandas qcut вам в помощь)
- для каждого дециля посчитать среднюю conversion

In [32]:
solo = SoloModel(CatBoostClassifier(n_estimators=1000, silent=True, learning_rate=0.01,
                                    cat_features=cat_features, random_state=10))

In [33]:
solo.fit(X_train, y_train, treat_train)

SoloModel(estimator=<catboost.core.CatBoostClassifier object at 0x000002C871C62110>)

In [34]:
uplift_solo = solo.predict(X_test)

In [35]:
df = pd.DataFrame()

In [36]:
df = pd.DataFrame({'true': y_test,
                  'treat': treat_test,
                  'uplift': uplift_solo})

In [37]:
df.sort_values('uplift', ascending=False, inplace=True)

In [38]:
data, bins = pd.qcut(df['uplift'], q=10, retbins=True)

In [39]:
df_decile = pd.DataFrame(
    {'percentile': [],
    'mean_conversion': []})
num = 0

for i in range(len(bins) - 1, 0, -1):
    decile = df.loc[(df['uplift'] <= bins[i]) & df['uplift'] > bins[i - 1], 'true'].mean()
    df_decile = pd.concat([df_decile, pd.DataFrame({'percentile': f'{num * 10} - {(num + 1 * 10)}',
                                                    'mean_conversion': [decile]})])
    num += 1

In [40]:
df_decile

percentile  mean_conversion
0     0 - 10         0.146771
0    10 - 11         0.138831
0    20 - 12         0.133325
0    30 - 13         0.130952
0    40 - 14         0.126736
0    50 - 15         0.124271
0    60 - 16         0.123682
0    70 - 17         0.108488
0    80 - 18         0.095052
0    90 - 19         0.072396